# lgbm-tfidf 도전

In [24]:
warnings.filterwarnings(action='ignore')
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer
import re

In [25]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline
import lightgbm as lgb
from matplotlib import pyplot as plt
from matplotlib import rcParams
import numpy as np
from pathlib import Path
import pandas as pd
from sklearn.metrics import accuracy_score
from sklearn.model_selection import StratifiedKFold
import seaborn as sns
import warnings
from nltk.tokenize import word_tokenize
from nltk.tokenize import WordPunctTokenizer
from nltk.tokenize.regexp import RegexpTokenizer
from nltk.stem.snowball import SnowballStemmer 
import nltk
from nltk.corpus import stopwords
from sklearn.metrics import accuracy_score, log_loss
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer, HashingVectorizer

In [26]:
trn = pd.read_csv('C:\\Users\\USER\\Desktop\\open\\train.csv', encoding = 'utf-8')
tst = pd.read_csv('C:\\Users\\USER\\Desktop\\open\\test_x.csv', encoding = 'utf-8')
sample_submission = pd.read_csv('C:\\Users\\USER\\Desktop\\open\\sample_submission.csv', encoding = 'utf-8')

[문자열 가공]  
1. Bag-of-Words(simple):countVectorizer, TfidVectorizer  
2. Hashing Trick: hashingVectorizer  
3. Embeddings(with 신경망): Word2Vec, GloVe

## Bag-of-Words_Count 피처 생성1

In [27]:
vec = CountVectorizer(tokenizer=word_tokenize, stop_words=stopwords.words('english'), ngram_range=(1, 3), min_df=100)
X_cnt = vec.fit_transform(trn['text'])
X_tst = vec.fit_transform(tst['text'])
print(X_cnt.shape, X_tst.shape)

(54879, 2892) (19617, 1990)


In [28]:
X_cnt[0, :50].todense()

matrix([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0]], dtype=int64)

In [30]:
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

## Lgbm 모델 학습_count

In [33]:
y=trn.author.values
X_cnt = X_cnt.astype('float32') 
y = y.astype('float32')
X_tst = X_tst.astype('float32')   
p = np.zeros((X_cnt.shape[0], 5))
p_tst = np.zeros((X_tst.shape[0], 5))
#(cv.split(),1)
# -  StratifiedKFold 경우, trn,y 독립과 종속(동일 분포를 위해) 인자
for i, (i_trn, i_val) in enumerate(cv.split(X_cnt, y), 1):
    print(f'training model for CV #{i}')
    clf = lgb.LGBMClassifier(objective='multiclass',
                             n_estimators=3000,
                             num_leaves=256,
                             learning_rate=0.009,
                             subsample=.8,
                             subsample_freq=1,
                             num_iterations=2000,
                             colsample_bytree=.6,
                             random_state=42,
                             n_jobs=-1)
    
    clf.fit(X_cnt[i_trn], y[i_trn],eval_set=[(X_cnt[i_val], y[i_val])], eval_metric='multiclass',early_stopping_rounds=15)
    p[i_val, :] = clf.predict_proba(X_cnt[i_val])
    #확률로 예측값 출력
    p_tst += clf.predict_proba(X_tst) / 5
    #평균 효과를 위해 먼저 폴드 개수만큼 나누어 초기화

training model for CV #1
[1]	valid_0's multi_logloss: 1.55939
Training until validation scores don't improve for 15 rounds
[2]	valid_0's multi_logloss: 1.54824
[3]	valid_0's multi_logloss: 1.53794
[4]	valid_0's multi_logloss: 1.52787
[5]	valid_0's multi_logloss: 1.51826
[6]	valid_0's multi_logloss: 1.50833
[7]	valid_0's multi_logloss: 1.49876
[8]	valid_0's multi_logloss: 1.48943
[9]	valid_0's multi_logloss: 1.48034
[10]	valid_0's multi_logloss: 1.47112
[11]	valid_0's multi_logloss: 1.46225
[12]	valid_0's multi_logloss: 1.45356
[13]	valid_0's multi_logloss: 1.44575
[14]	valid_0's multi_logloss: 1.43761
[15]	valid_0's multi_logloss: 1.42952
[16]	valid_0's multi_logloss: 1.4214
[17]	valid_0's multi_logloss: 1.41366
[18]	valid_0's multi_logloss: 1.40599
[19]	valid_0's multi_logloss: 1.39846
[20]	valid_0's multi_logloss: 1.39101
[21]	valid_0's multi_logloss: 1.38326
[22]	valid_0's multi_logloss: 1.37611
[23]	valid_0's multi_logloss: 1.36894
[24]	valid_0's multi_logloss: 1.36176
[25]	valid_0

[210]	valid_0's multi_logloss: 0.849276
[211]	valid_0's multi_logloss: 0.848168
[212]	valid_0's multi_logloss: 0.847028
[213]	valid_0's multi_logloss: 0.845886
[214]	valid_0's multi_logloss: 0.844742
[215]	valid_0's multi_logloss: 0.84366
[216]	valid_0's multi_logloss: 0.842571
[217]	valid_0's multi_logloss: 0.841456
[218]	valid_0's multi_logloss: 0.840341
[219]	valid_0's multi_logloss: 0.839218
[220]	valid_0's multi_logloss: 0.838215
[221]	valid_0's multi_logloss: 0.837141
[222]	valid_0's multi_logloss: 0.836113
[223]	valid_0's multi_logloss: 0.83497
[224]	valid_0's multi_logloss: 0.83396
[225]	valid_0's multi_logloss: 0.833002
[226]	valid_0's multi_logloss: 0.831999
[227]	valid_0's multi_logloss: 0.830953
[228]	valid_0's multi_logloss: 0.829918
[229]	valid_0's multi_logloss: 0.828955
[230]	valid_0's multi_logloss: 0.827927
[231]	valid_0's multi_logloss: 0.826883
[232]	valid_0's multi_logloss: 0.825871
[233]	valid_0's multi_logloss: 0.824903
[234]	valid_0's multi_logloss: 0.823984
[23

[416]	valid_0's multi_logloss: 0.705618
[417]	valid_0's multi_logloss: 0.705175
[418]	valid_0's multi_logloss: 0.704705
[419]	valid_0's multi_logloss: 0.704342
[420]	valid_0's multi_logloss: 0.703914
[421]	valid_0's multi_logloss: 0.703507
[422]	valid_0's multi_logloss: 0.703137
[423]	valid_0's multi_logloss: 0.702743
[424]	valid_0's multi_logloss: 0.702361
[425]	valid_0's multi_logloss: 0.701961
[426]	valid_0's multi_logloss: 0.701535
[427]	valid_0's multi_logloss: 0.701114
[428]	valid_0's multi_logloss: 0.700793
[429]	valid_0's multi_logloss: 0.700445
[430]	valid_0's multi_logloss: 0.700071
[431]	valid_0's multi_logloss: 0.699699
[432]	valid_0's multi_logloss: 0.699283
[433]	valid_0's multi_logloss: 0.698904
[434]	valid_0's multi_logloss: 0.698501
[435]	valid_0's multi_logloss: 0.698096
[436]	valid_0's multi_logloss: 0.69768
[437]	valid_0's multi_logloss: 0.697247
[438]	valid_0's multi_logloss: 0.696835
[439]	valid_0's multi_logloss: 0.69645
[440]	valid_0's multi_logloss: 0.696099
[4

[622]	valid_0's multi_logloss: 0.649197
[623]	valid_0's multi_logloss: 0.649014
[624]	valid_0's multi_logloss: 0.648863
[625]	valid_0's multi_logloss: 0.648722
[626]	valid_0's multi_logloss: 0.648549
[627]	valid_0's multi_logloss: 0.648396
[628]	valid_0's multi_logloss: 0.648204
[629]	valid_0's multi_logloss: 0.648054
[630]	valid_0's multi_logloss: 0.647924
[631]	valid_0's multi_logloss: 0.647769
[632]	valid_0's multi_logloss: 0.647621
[633]	valid_0's multi_logloss: 0.64749
[634]	valid_0's multi_logloss: 0.647323
[635]	valid_0's multi_logloss: 0.647164
[636]	valid_0's multi_logloss: 0.646982
[637]	valid_0's multi_logloss: 0.646814
[638]	valid_0's multi_logloss: 0.646664
[639]	valid_0's multi_logloss: 0.64651
[640]	valid_0's multi_logloss: 0.646346
[641]	valid_0's multi_logloss: 0.646174
[642]	valid_0's multi_logloss: 0.645972
[643]	valid_0's multi_logloss: 0.645829
[644]	valid_0's multi_logloss: 0.645632
[645]	valid_0's multi_logloss: 0.645477
[646]	valid_0's multi_logloss: 0.645329
[6

[828]	valid_0's multi_logloss: 0.62565
[829]	valid_0's multi_logloss: 0.625582
[830]	valid_0's multi_logloss: 0.625522
[831]	valid_0's multi_logloss: 0.625455
[832]	valid_0's multi_logloss: 0.625428
[833]	valid_0's multi_logloss: 0.625355
[834]	valid_0's multi_logloss: 0.625279
[835]	valid_0's multi_logloss: 0.62525
[836]	valid_0's multi_logloss: 0.625198
[837]	valid_0's multi_logloss: 0.625128
[838]	valid_0's multi_logloss: 0.625071
[839]	valid_0's multi_logloss: 0.624999
[840]	valid_0's multi_logloss: 0.624933
[841]	valid_0's multi_logloss: 0.624893
[842]	valid_0's multi_logloss: 0.624839
[843]	valid_0's multi_logloss: 0.62478
[844]	valid_0's multi_logloss: 0.624758
[845]	valid_0's multi_logloss: 0.624701
[846]	valid_0's multi_logloss: 0.624661
[847]	valid_0's multi_logloss: 0.624612
[848]	valid_0's multi_logloss: 0.624537
[849]	valid_0's multi_logloss: 0.624464
[850]	valid_0's multi_logloss: 0.624392
[851]	valid_0's multi_logloss: 0.624353
[852]	valid_0's multi_logloss: 0.624275
[85

[1033]	valid_0's multi_logloss: 0.616826
[1034]	valid_0's multi_logloss: 0.616798
[1035]	valid_0's multi_logloss: 0.616789
[1036]	valid_0's multi_logloss: 0.61676
[1037]	valid_0's multi_logloss: 0.616743
[1038]	valid_0's multi_logloss: 0.616721
[1039]	valid_0's multi_logloss: 0.616704
[1040]	valid_0's multi_logloss: 0.616668
[1041]	valid_0's multi_logloss: 0.616645
[1042]	valid_0's multi_logloss: 0.616638
[1043]	valid_0's multi_logloss: 0.616612
[1044]	valid_0's multi_logloss: 0.616592
[1045]	valid_0's multi_logloss: 0.616579
[1046]	valid_0's multi_logloss: 0.616592
[1047]	valid_0's multi_logloss: 0.61656
[1048]	valid_0's multi_logloss: 0.616535
[1049]	valid_0's multi_logloss: 0.616512
[1050]	valid_0's multi_logloss: 0.616506
[1051]	valid_0's multi_logloss: 0.616478
[1052]	valid_0's multi_logloss: 0.616475
[1053]	valid_0's multi_logloss: 0.616458
[1054]	valid_0's multi_logloss: 0.61643
[1055]	valid_0's multi_logloss: 0.616406
[1056]	valid_0's multi_logloss: 0.616375
[1057]	valid_0's mu

ValueError: Number of features of the model must match the input. Model n_features_ is 2892 and input n_features is 1990 

In [34]:
print(f'Accuracy (CV): {accuracy_score(y, np.argmax(p, axis=1)) * 100:8.4f}%')
print(f'Log Loss (CV): {log_loss(pd.get_dummies(y), p):8.4f}')

Accuracy (CV):  34.5542%
Log Loss (CV):   1.4107


## Bag-of-Words_Tfidf 피처 생성2

In [36]:
vec = TfidfVectorizer(tokenizer=word_tokenize, stop_words=stopwords.words('english'), ngram_range=(1, 3), min_df=50)
X_tfi = vec.fit_transform(trn['text'])
X_tst = vec.transform(tst['text'])
print(X_tfi.shape, X_tst.shape)

(54879, 5897) (19617, 5897)


In [37]:
X_tfi[0, :50].todense()

matrix([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0.]])

In [39]:
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

## Lgbm 모델 학습_tfidf

In [41]:
y=trn.author.values
X_tfi = X_tfi.astype('float32') 
y = y.astype('float32')
X_tst = X_tst.astype('float32')
p = np.zeros((X_tfi.shape[0], 5))
p_tst = np.zeros((X_tst.shape[0], 5))
#(cv.split(),1)
# -  StratifiedKFold 경우, trn,y 독립과 종속(동일 분포를 위해) 인자
for i, (i_trn, i_val) in enumerate(cv.split(X_tfi, y), 1):
    print(f'training model for CV #{i}')
    clf = lgb.LGBMClassifier(objective='multiclass',
                             n_estimators=3000,
                             num_leaves=256,
                             learning_rate=0.009,
                             subsample=.8,
                             subsample_freq=1,
                             num_iterations=2000,
                             colsample_bytree=.6,
                             random_state=42,
                             n_jobs=-1)
    
    clf.fit(X_tfi[i_trn], y[i_trn],eval_set=[(X_tfi[i_val], y[i_val])], eval_metric='multiclass',early_stopping_rounds=15)
    p[i_val, :] = clf.predict_proba(X_tfi[i_val])
    #확률로 예측값 출력
    p_tst += clf.predict_proba(X_tst) / 5
    #평균 효과를 위해 먼저 폴드 개수만큼 나누어 초기화

training model for CV #1
[1]	valid_0's multi_logloss: 1.55868
Training until validation scores don't improve for 15 rounds
[2]	valid_0's multi_logloss: 1.54782
[3]	valid_0's multi_logloss: 1.53737
[4]	valid_0's multi_logloss: 1.52753
[5]	valid_0's multi_logloss: 1.51746
[6]	valid_0's multi_logloss: 1.5075
[7]	valid_0's multi_logloss: 1.49813
[8]	valid_0's multi_logloss: 1.4884
[9]	valid_0's multi_logloss: 1.47915
[10]	valid_0's multi_logloss: 1.47006
[11]	valid_0's multi_logloss: 1.46116
[12]	valid_0's multi_logloss: 1.45214
[13]	valid_0's multi_logloss: 1.44358
[14]	valid_0's multi_logloss: 1.43538
[15]	valid_0's multi_logloss: 1.4273
[16]	valid_0's multi_logloss: 1.41916
[17]	valid_0's multi_logloss: 1.41121
[18]	valid_0's multi_logloss: 1.4039
[19]	valid_0's multi_logloss: 1.39589
[20]	valid_0's multi_logloss: 1.38813
[21]	valid_0's multi_logloss: 1.38071
[22]	valid_0's multi_logloss: 1.37352
[23]	valid_0's multi_logloss: 1.3663
[24]	valid_0's multi_logloss: 1.35944
[25]	valid_0's m

[210]	valid_0's multi_logloss: 0.841545
[211]	valid_0's multi_logloss: 0.84051
[212]	valid_0's multi_logloss: 0.83947
[213]	valid_0's multi_logloss: 0.838285
[214]	valid_0's multi_logloss: 0.837101
[215]	valid_0's multi_logloss: 0.83595
[216]	valid_0's multi_logloss: 0.834806
[217]	valid_0's multi_logloss: 0.83375
[218]	valid_0's multi_logloss: 0.832471
[219]	valid_0's multi_logloss: 0.831357
[220]	valid_0's multi_logloss: 0.830372
[221]	valid_0's multi_logloss: 0.829387
[222]	valid_0's multi_logloss: 0.828246
[223]	valid_0's multi_logloss: 0.827163
[224]	valid_0's multi_logloss: 0.826154
[225]	valid_0's multi_logloss: 0.825064
[226]	valid_0's multi_logloss: 0.824021
[227]	valid_0's multi_logloss: 0.823051
[228]	valid_0's multi_logloss: 0.822104
[229]	valid_0's multi_logloss: 0.821159
[230]	valid_0's multi_logloss: 0.82011
[231]	valid_0's multi_logloss: 0.819154
[232]	valid_0's multi_logloss: 0.818158
[233]	valid_0's multi_logloss: 0.817202
[234]	valid_0's multi_logloss: 0.816174
[235]

[416]	valid_0's multi_logloss: 0.693119
[417]	valid_0's multi_logloss: 0.692685
[418]	valid_0's multi_logloss: 0.69226
[419]	valid_0's multi_logloss: 0.691832
[420]	valid_0's multi_logloss: 0.69135
[421]	valid_0's multi_logloss: 0.690906
[422]	valid_0's multi_logloss: 0.690562
[423]	valid_0's multi_logloss: 0.690144
[424]	valid_0's multi_logloss: 0.689751
[425]	valid_0's multi_logloss: 0.68932
[426]	valid_0's multi_logloss: 0.688929
[427]	valid_0's multi_logloss: 0.688482
[428]	valid_0's multi_logloss: 0.688097
[429]	valid_0's multi_logloss: 0.68775
[430]	valid_0's multi_logloss: 0.687352
[431]	valid_0's multi_logloss: 0.686936
[432]	valid_0's multi_logloss: 0.68654
[433]	valid_0's multi_logloss: 0.686178
[434]	valid_0's multi_logloss: 0.685775
[435]	valid_0's multi_logloss: 0.685409
[436]	valid_0's multi_logloss: 0.685025
[437]	valid_0's multi_logloss: 0.684624
[438]	valid_0's multi_logloss: 0.684243
[439]	valid_0's multi_logloss: 0.68393
[440]	valid_0's multi_logloss: 0.683519
[441]	

[622]	valid_0's multi_logloss: 0.637867
[623]	valid_0's multi_logloss: 0.637704
[624]	valid_0's multi_logloss: 0.637508
[625]	valid_0's multi_logloss: 0.637367
[626]	valid_0's multi_logloss: 0.637232
[627]	valid_0's multi_logloss: 0.637083
[628]	valid_0's multi_logloss: 0.636926
[629]	valid_0's multi_logloss: 0.636782
[630]	valid_0's multi_logloss: 0.636605
[631]	valid_0's multi_logloss: 0.636464
[632]	valid_0's multi_logloss: 0.63632
[633]	valid_0's multi_logloss: 0.636151
[634]	valid_0's multi_logloss: 0.635991
[635]	valid_0's multi_logloss: 0.635844
[636]	valid_0's multi_logloss: 0.635689
[637]	valid_0's multi_logloss: 0.635534
[638]	valid_0's multi_logloss: 0.635375
[639]	valid_0's multi_logloss: 0.635186
[640]	valid_0's multi_logloss: 0.63503
[641]	valid_0's multi_logloss: 0.634861
[642]	valid_0's multi_logloss: 0.634703
[643]	valid_0's multi_logloss: 0.634584
[644]	valid_0's multi_logloss: 0.634432
[645]	valid_0's multi_logloss: 0.634289
[646]	valid_0's multi_logloss: 0.63415
[64

[828]	valid_0's multi_logloss: 0.61777
[829]	valid_0's multi_logloss: 0.61769
[830]	valid_0's multi_logloss: 0.617627
[831]	valid_0's multi_logloss: 0.617533
[832]	valid_0's multi_logloss: 0.617535
[833]	valid_0's multi_logloss: 0.617482
[834]	valid_0's multi_logloss: 0.617433
[835]	valid_0's multi_logloss: 0.617433
[836]	valid_0's multi_logloss: 0.617439
[837]	valid_0's multi_logloss: 0.617379
[838]	valid_0's multi_logloss: 0.617338
[839]	valid_0's multi_logloss: 0.617304
[840]	valid_0's multi_logloss: 0.617271
[841]	valid_0's multi_logloss: 0.617203
[842]	valid_0's multi_logloss: 0.617162
[843]	valid_0's multi_logloss: 0.61713
[844]	valid_0's multi_logloss: 0.617138
[845]	valid_0's multi_logloss: 0.617118
[846]	valid_0's multi_logloss: 0.617086
[847]	valid_0's multi_logloss: 0.617058
[848]	valid_0's multi_logloss: 0.617011
[849]	valid_0's multi_logloss: 0.616962
[850]	valid_0's multi_logloss: 0.616908
[851]	valid_0's multi_logloss: 0.616881
[852]	valid_0's multi_logloss: 0.616815
[85

...

[118]	valid_0's multi_logloss: 0.983984
[119]	valid_0's multi_logloss: 0.981682
[120]	valid_0's multi_logloss: 0.979554
[121]	valid_0's multi_logloss: 0.977415
[122]	valid_0's multi_logloss: 0.975256
[123]	valid_0's multi_logloss: 0.973179
[124]	valid_0's multi_logloss: 0.970991
[125]	valid_0's multi_logloss: 0.968875
[126]	valid_0's multi_logloss: 0.966691
[127]	valid_0's multi_logloss: 0.964621
[128]	valid_0's multi_logloss: 0.962519
[129]	valid_0's multi_logloss: 0.960541
[130]	valid_0's multi_logloss: 0.958465
[131]	valid_0's multi_logloss: 0.956305
[132]	valid_0's multi_logloss: 0.954391
[133]	valid_0's multi_logloss: 0.952544
[134]	valid_0's multi_logloss: 0.950561
[135]	valid_0's multi_logloss: 0.948484
[136]	valid_0's multi_logloss: 0.946448
[137]	valid_0's multi_logloss: 0.944331
[138]	valid_0's multi_logloss: 0.942422
[139]	valid_0's multi_logloss: 0.940504
[140]	valid_0's multi_logloss: 0.938635
[141]	valid_0's multi_logloss: 0.936698
[142]	valid_0's multi_logloss: 0.934861


[324]	valid_0's multi_logloss: 0.737239
[325]	valid_0's multi_logloss: 0.736637
[326]	valid_0's multi_logloss: 0.735965
[327]	valid_0's multi_logloss: 0.735353
[328]	valid_0's multi_logloss: 0.734712
[329]	valid_0's multi_logloss: 0.734108
[330]	valid_0's multi_logloss: 0.733466
[331]	valid_0's multi_logloss: 0.732836
[332]	valid_0's multi_logloss: 0.732174
[333]	valid_0's multi_logloss: 0.731472
[334]	valid_0's multi_logloss: 0.730795
[335]	valid_0's multi_logloss: 0.730172
[336]	valid_0's multi_logloss: 0.729564
[337]	valid_0's multi_logloss: 0.728937
[338]	valid_0's multi_logloss: 0.72832
[339]	valid_0's multi_logloss: 0.727688
[340]	valid_0's multi_logloss: 0.727027
[341]	valid_0's multi_logloss: 0.726433
[342]	valid_0's multi_logloss: 0.725838
[343]	valid_0's multi_logloss: 0.725182
[344]	valid_0's multi_logloss: 0.724567
[345]	valid_0's multi_logloss: 0.723913
[346]	valid_0's multi_logloss: 0.723261
[347]	valid_0's multi_logloss: 0.722692
[348]	valid_0's multi_logloss: 0.722207
[

[530]	valid_0's multi_logloss: 0.651367
[531]	valid_0's multi_logloss: 0.651121
[532]	valid_0's multi_logloss: 0.650848
[533]	valid_0's multi_logloss: 0.650631
[534]	valid_0's multi_logloss: 0.650408
[535]	valid_0's multi_logloss: 0.650176
[536]	valid_0's multi_logloss: 0.649943
[537]	valid_0's multi_logloss: 0.649734
[538]	valid_0's multi_logloss: 0.649472
[539]	valid_0's multi_logloss: 0.64926
[540]	valid_0's multi_logloss: 0.649036
[541]	valid_0's multi_logloss: 0.648795
[542]	valid_0's multi_logloss: 0.648558
[543]	valid_0's multi_logloss: 0.648314
[544]	valid_0's multi_logloss: 0.648077
[545]	valid_0's multi_logloss: 0.647873
[546]	valid_0's multi_logloss: 0.647698
[547]	valid_0's multi_logloss: 0.647456
[548]	valid_0's multi_logloss: 0.64722
[549]	valid_0's multi_logloss: 0.647027
[550]	valid_0's multi_logloss: 0.646773
[551]	valid_0's multi_logloss: 0.646578
[552]	valid_0's multi_logloss: 0.646385
[553]	valid_0's multi_logloss: 0.646159
[554]	valid_0's multi_logloss: 0.645972
[5

[736]	valid_0's multi_logloss: 0.62134
[737]	valid_0's multi_logloss: 0.62121
[738]	valid_0's multi_logloss: 0.621129
[739]	valid_0's multi_logloss: 0.621017
[740]	valid_0's multi_logloss: 0.62096
[741]	valid_0's multi_logloss: 0.6209
[742]	valid_0's multi_logloss: 0.620874
[743]	valid_0's multi_logloss: 0.620825
[744]	valid_0's multi_logloss: 0.620731
[745]	valid_0's multi_logloss: 0.62066
[746]	valid_0's multi_logloss: 0.620595
[747]	valid_0's multi_logloss: 0.62053
[748]	valid_0's multi_logloss: 0.620443
[749]	valid_0's multi_logloss: 0.620392
[750]	valid_0's multi_logloss: 0.620305
[751]	valid_0's multi_logloss: 0.620225
[752]	valid_0's multi_logloss: 0.6202
[753]	valid_0's multi_logloss: 0.62009
[754]	valid_0's multi_logloss: 0.620029
[755]	valid_0's multi_logloss: 0.619912
[756]	valid_0's multi_logloss: 0.619799
[757]	valid_0's multi_logloss: 0.619714
[758]	valid_0's multi_logloss: 0.619672
[759]	valid_0's multi_logloss: 0.6196
[760]	valid_0's multi_logloss: 0.619513
[761]	valid_

[942]	valid_0's multi_logloss: 0.613253
[943]	valid_0's multi_logloss: 0.613258
[944]	valid_0's multi_logloss: 0.613227
[945]	valid_0's multi_logloss: 0.613186
[946]	valid_0's multi_logloss: 0.613181
[947]	valid_0's multi_logloss: 0.613167
[948]	valid_0's multi_logloss: 0.613183
[949]	valid_0's multi_logloss: 0.613171
[950]	valid_0's multi_logloss: 0.613179
[951]	valid_0's multi_logloss: 0.613145
[952]	valid_0's multi_logloss: 0.613132
[953]	valid_0's multi_logloss: 0.613129
[954]	valid_0's multi_logloss: 0.613145
[955]	valid_0's multi_logloss: 0.613094
[956]	valid_0's multi_logloss: 0.613113
[957]	valid_0's multi_logloss: 0.613091
[958]	valid_0's multi_logloss: 0.613077
[959]	valid_0's multi_logloss: 0.613063
[960]	valid_0's multi_logloss: 0.613072
[961]	valid_0's multi_logloss: 0.613079
[962]	valid_0's multi_logloss: 0.613101
[963]	valid_0's multi_logloss: 0.613099
[964]	valid_0's multi_logloss: 0.613097
[965]	valid_0's multi_logloss: 0.613087
[966]	valid_0's multi_logloss: 0.613074


In [42]:
print(f'Accuracy (CV): {accuracy_score(y, np.argmax(p, axis=1)) * 100:8.4f}%')
print(f'Log Loss (CV): {log_loss(pd.get_dummies(y), p):8.4f}')

Accuracy (CV):  76.9274%
Log Loss (CV):   0.6212


In [15]:
df = pd.concat([train, test], axis=0)

In [16]:
print(df.shape)
df.tail()

(74496, 3)


,index,text,author
19612,19612,"At the end of another day or two, odin growing...",NaN
19613,19613,"All afternoon we sat together, mostly in silen...",NaN
19614,19614,"odin, having carried his thanks to odin, proc...",NaN
19615,19615,"Soon after this, upon odin's leaving the room,...",NaN
19616,19616,"And all the worse for the doomed man, that the...",NaN


In [17]:
df.fillna(-1, inplace=True)
df.tail()

,index,text,author
19612,19612,"At the end of another day or two, odin growing...",-1.0
19613,19613,"All afternoon we sat together, mostly in silen...",-1.0
19614,19614,"odin, having carried his thanks to odin, proc...",-1.0
19615,19615,"Soon after this, upon odin's leaving the room,...",-1.0
19616,19616,"And all the worse for the doomed man, that the...",-1.0


In [18]:
#부호를 제거해주는 함수
def alpha_num(text):
    return re.sub(r'[^A-Za-z0-9 ]', '', text)

df['text']=df['text'].apply(alpha_num)

In [19]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\USER\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [20]:
df=df['text'].values

In [21]:
for i, document in enumerate(df):
    words = word_tokenize(document)
    #print(words) #
    clean_words = []
    for word in words:
        word = word.lower()                         #대소분자 변환
        if word not in stopwords.words('english'): #불용어 제거
            stemmer = SnowballStemmer('english')
            word = stemmer.stem(word)              #어간 추출
            clean_words.append(word)
    document = ' '.join(clean_words)
    df[i] = document

KeyboardInterrupt: 

In [ ]:
df = CountVectorizer(tokenizer=word_tokenize, stop_words='english', min_df=50).fit_transform(df)

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.pipeline import Pipeline

vectorizer = CountVectorizer(analyzer = 'word', 
                             tokenizer = None,
                             preprocessor = None, 
                             stop_words = None, 
                             min_df = 2, # 토큰이 나타날 최소 문서 개수
                             ngram_range=(4, 4),
                             max_features = 20000
                            )
vectorizer
pipeline = Pipeline([
    ('vect', vectorizer),
])


%time train_data_features = pipeline.fit_transform(df_train['review_clean'])
%time test_data_features = pipeline.fit_transform(df_test['review_clean'])

In [ ]:
# train test 분리
y_train = df['author'].values

In [ ]:
x_train = df[:54879]                #train을 위한 독립 320000개     
x_test = df[54879:]              #test을 위한 독립 80000개

In [ ]:
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

In [ ]:
x_train

In [ ]:
x_test

In [ ]:
y_train=y_train[:54879]

In [ ]:
x_train = x_train.astype('float32') 
y_train = y_train.astype('float32')
x_test = x_test.astype('float32')   

In [ ]:
p_val = np.zeros((x_train.shape[0], 5))
p_tst = np.zeros((x_test.shape[0], 5))
#(cv.split(),1)
# -  StratifiedKFold 경우, trn,y 독립과 종속(동일 분포를 위해) 인자
for i, (i_trn, i_val) in enumerate(cv.split(x_train, y_train), 1):
    print(f'training model for CV #{i}')
    clf = lgb.LGBMClassifier(objective='multiclass',
                             n_estimators=3000,
                             num_leaves=256,
                             learning_rate=0.009,
                             subsample=.8,
                             subsample_freq=1,
                             num_iterations=2000,
                             colsample_bytree=.6,
                             random_state=42,
                             n_jobs=-1)
    
    clf.fit(x_train[i_trn], y_train[i_trn],eval_set=[(x_train[i_val], y_train[i_val])], eval_metric='multiclass',early_stopping_rounds=15)
    
    p_val[i_val, :] = clf.predict_proba(x_train[i_val])
    #확률로 예측값 출력
    p_tst += clf.predict_proba(x_test) / 5
    #평균 효과를 위해 먼저 폴드 개수만큼 나누어 초기화

In [ ]:
p_tst

In [ ]:
print(f'Log loss (CV) : {log_loss(pd.get_dummies(y_train),p_tst):8.4f}')

In [ ]:
print(f'Log loss (CV) : {log_loss(pd.get_dummies(y_train),p_val):8.4f}')

In [43]:
# submission
sample_submission[['0','1','2','3','4']] = p_tst
sample_submission

,index,0,1,2,3,4
0,0,0.036046,0.786655,0.134206,0.037717,0.005376
1,1,0.012631,0.911046,0.009961,0.015190,0.051172
2,2,0.975920,0.003870,0.006276,0.002627,0.011307
3,3,0.005586,0.006153,0.969737,0.013309,0.005215
4,4,0.590571,0.101196,0.126925,0.107833,0.073474
...,...,...,...,...,...,...
19612,19612,0.008478,0.988235,0.000196,0.002016,0.001075
19613,19613,0.200611,0.004116,0.013317,0.015521,0.766435
19614,19614,0.006986,0.991535,0.000125,0.000973,0.000381
19615,19615,0.035062,0.840682,0.038176,0.077185,0.008895


In [44]:
sample_submission.to_csv('thisissubmission.csv', index = False, encoding = 'utf-8')